<a href="https://colab.research.google.com/github/SangamSilwal/DeepLearning_DSeries/blob/main/T9_RNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/content/100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
#tokrnize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [4]:
tokenize("what is the name of 'san' gam? ")

['what', 'is', 'the', 'name', 'of', 'san', 'gam']

In [5]:
#vocab
vocab = {'<UNK>':0}

In [6]:
def  build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [7]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [8]:
len(vocab)

324

In [9]:
#converting words to numerical indices
def text_to_indices(text,vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [12]:
text_to_indices("What is campus",vocab)

[1, 2, 0]

In [13]:
import torch
from torch.utils.data import Dataset,DataLoader

In [14]:
class QADataset(Dataset):

  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numerical_question),torch.tensor(numerical_answer)


In [15]:
dataset = QADataset(df,vocab)
dataloader = DataLoader(dataset,batch_size = 1,shuffle=True)

In [24]:
for question,answer in dataloader:
  print(question,answer[0])

tensor([[  1,   2,   3,   4,   5, 279]]) tensor([280])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([28])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([110])
tensor([[ 42, 255,   2, 256,  83, 257, 258]]) tensor([259])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([36])
tensor([[  1,   2,   3,  37, 133,   5,  26]]) tensor([134])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([215])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([85])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([136])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([95])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([184])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([36])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[10, 75, 76]]) tensor([77])
tensor([[ 42,  18,   2,   3, 281,  12,   

In [25]:
import torch.nn as nn

In [38]:
class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50,64,batch_first=True)
    self.fc = nn.Linear(64,vocab_size)

  def forward(self,question):
    embedded_question = self.embedding(question)
    hidden,final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))
    return output

In [65]:
learning_rate = 0.001
epochs = 20

In [66]:
model = SimpleRNN(len(vocab))

In [67]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [68]:
for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 523.342126
Epoch: 2, Loss: 451.601398
Epoch: 3, Loss: 372.103817
Epoch: 4, Loss: 314.198555
Epoch: 5, Loss: 264.232570
Epoch: 6, Loss: 217.828214
Epoch: 7, Loss: 174.301284
Epoch: 8, Loss: 136.928656
Epoch: 9, Loss: 106.579581
Epoch: 10, Loss: 82.153619
Epoch: 11, Loss: 64.627872
Epoch: 12, Loss: 51.037096
Epoch: 13, Loss: 41.069220
Epoch: 14, Loss: 33.659059
Epoch: 15, Loss: 28.429530
Epoch: 16, Loss: 23.642810
Epoch: 17, Loss: 20.266184
Epoch: 18, Loss: 17.226203
Epoch: 19, Loss: 15.077315
Epoch: 20, Loss: 13.149023


In [76]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)
  print(question_tensor)

  # send to model
  output = model(question_tensor)
  print(output)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)
  print(probs)

  # find index of max prob
  value, index = torch.max(probs, dim=1)
  print(value,index)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [77]:
predict(model, "What is the largest planet in our solar system?")

tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]])
tensor([[-3.5433e+00, -3.6609e+00, -3.7333e+00, -2.7613e+00, -3.3607e+00,
         -3.9278e+00, -9.4704e-01, -1.5595e+00, -3.1938e+00, -5.8183e-01,
         -2.8891e+00, -2.9719e+00, -3.2520e+00, -3.7033e+00, -3.2686e+00,
         -3.6284e+00, -2.0219e-02, -3.1440e+00, -3.2193e+00, -3.1960e+00,
         -3.5169e+00, -3.9905e+00, -3.3744e+00,  7.1518e+00, -3.2246e+00,
         -3.0984e+00, -3.5253e+00, -3.9704e+00,  2.0735e+00, -3.6168e+00,
         -2.7994e+00, -3.8050e+00, -1.9800e+00, -3.2687e+00, -3.3006e+00,
         -3.3363e+00,  3.7015e+00, -3.2656e+00, -3.7809e+00, -3.1513e+00,
         -3.1890e+00, -5.2196e-02, -3.3087e+00, -2.9668e+00, -3.2827e+00,
         -3.0915e+00, -3.6971e+00, -3.8991e+00, -3.1119e+00,  1.1796e-01,
         -3.3508e+00, -3.5861e+00,  3.0900e-01,  1.8029e+00, -7.0532e-02,
         -3.6095e+00, -3.6005e+00, -2.8469e+00, -1.7461e+00, -3.0026e+00,
         -3.0612e+00, -8.3900e-01, -2.9772e+00, -3.3421e+00, -3.0